In [1]:
"""
Routine to process data of a particle with slip condition using rigid nultiblob method.                 
@author: Nicolas Moreno - nmoreno@bcamath.org                               
"""

'\nRoutine to process data of a particle with slip condition using rigid nultiblob method.                 \n@author: Nicolas Moreno - nmoreno@bcamath.org                               \n'

In [2]:
##Import some basic python libraries 
import pandas as pd
import numpy as np
import os, sys, re
from matplotlib import pyplot as plt
##import pylab as plt

##Python script to generate mesh
sys.path.append('./sidmanager')
import createVirusPoints as cvp

createVirus: Running from another script


In [3]:
# Definition of useful functions 
#Functions to separate a 6x6 matrix dat into three sub 3x3 matrix. t: translation, r:rotational, c: coupling
def M(dat):
    mt = dat.values[0:3,0:3]
    mr = dat.values[3:,3:]
    mc = dat.values[0:3,3:]
    return mt,mr,mc

def K(dat):
    kt = dat[0:3,0:3]
    kr = dat[3:,3:]
    kc = dat[0:3,3:]
    return kt,kr,kc



In [4]:
# Definition of variables or parameters to study
sids = [2] #simulation id, this is the identifier for each simulation. For example 4 simualations names 1-4
Rs = [1]
rosinit  = [0.2]
resol = 0.069
coarseIcosphere = [12,42,162,642,2562,10242]  ##Number of points per icosaedrha scale finitely with these sequence

rcovid = np.array([48.0, 43.0, 33.0])
lspike = 25.0
rspike = 15.0

lspike = np.round(lspike/rcovid[0],1)
rspike = np.round(rspike/rcovid[0],1)
rcovid = np.round(rcovid/rcovid[0],1)


rcovid,lspike,rspike, lspike/resol, rspike/resol

(array([1. , 0.9, 0.7]), 0.5, 0.3, 7.246376811594202, 4.3478260869565215)

In [5]:
# Running the structure generating script
ros = []
for i in range(0,len(sids)):
    
    oFile = 'RigidMultiblobsWall/multi_bodies/Structures/virus%s' % sids[i]
    R  = Rs[i]      # Radius of the core/capsid
    ro = rosinit[i] # Distance between points of the structure. If isosurface for shell, ro is recomputed 
                    # in general leading to same or smaller ro
    RP = R          # Outer radius to start growing viral spikes
    rop = 5         # ratio between spikes distance rspikes and ro. (this define density of spikes). rspikes = ro*rop
                    # If using distribution 0 or 1 is used to define the discrete set of points where spikes can be
                    # located. It is needed to take into account that number of equidistant points in a sphere scale 
                    # discretely 12,42,162,642,2562,10242. Depending on the ratio between R and ro.
    nspikes = 0     # number of spikes defined directly for random distributed spikes otherwise rop is used
    pType =  0      # spike type 0: none, 1: rod, 2: rod-tetra, 3: rod-sphere, 4: sphere, 5: tetra mesh, 6:coarse tetra only 4 particles per spike
    ps = 0          # radius of spike / R. When using rod-spheres or rodtetrahedra. Given by ps*ro (or ro_recomputed)
    shell = 0      # flag to change the type of capsid. Dense or shell only. 
                   # 0 or 2 : only shell with equidistant points. 0: also creates randomly filled core
                   # 1 or 3 for filled core. 3: equidistant points in core, 1: random distributed filled core 
    loc = 0.5      # location of the center of mass of the virus loc = position cm / size box
    lp  = 0.5      # ratio spike lenght/R (number of points) of the spike
    eps1 = 2       # wavelength 1 for deformed sphere
    eps2 = 5       # wavelength 2 for deformed sphere
    deform=False   # Deformed core
    dist =0        # Distribution of spikes around core. 0: homogeneous, 1: random distributed on discrete mesh
                                                                     # 2: random in an outer shell (no discrete)
    patchFrac=0
    lp2 = 0
    simType='xyz'  #Type of file - xyz or colloid or sdpd. xyz creates both xyz and dpd file. whereas colloid and sdpd only those
                   #visualize data files using ovito to verify shapes.

    genBlender = True   ## if the xyz file to use for blender rendering need to be generated
                        ## If sdpdm the following parameters are needed 
    lbox = 5            # size of the periodic box for sdpd simulations
    bandid = 0  
    ellips = False      #TRue for covid like core shape
    rhoN = np.round(1./ro**3,0) #particle density of dpd/sdpd simulations 
    m = ro**3                   #mass of the particle for dpd/sdpd simulations
    masses = '%s,%s,%s,%s' %(m,m,m,m)
    xi = 1
    alpharange = 0
    data = cvp.dataFile(oFile, 3,lbox, R,RP,masses, rhoN, bandid, ellips, ro,rop,nspikes,lp,ps,deform,eps1,eps2,xi,alpharange, 1, 1., 1, "1.0, 0.0, 0.0", patchType=pType,shellOnly=shell,location=loc,ks=0,simtype=simType,distribution=dist, blender=genBlender, patchFrac=patchFrac, lp2=lp2)
    data.atoms()
    ros.append(data.ro)
    dFile = 'RigidMultiblobsWall/multi_bodies/data/info%s' % sids[i]
    f=open(dFile,"w")
    f.write("ro %s\n" %data.ro)
    f.write("nspikes %s\n" %nspikes)
    f.write("Rp %s\n" %RP)
    f.write("ps %s\n" %ps)
    f.write("shell %s\n" %shell)
    f.write("loc %s\n" %loc)
    f.write("lp %s\n" %lp)
    f.write("ellips %s\n" %ellips)
    f.write("slip_length %s\n" %xi)
    f.write("angle_range %s\n" %alpharange)


    f.close()
ros = np.array(ros)

1 number of solvent types
gen core
Current ro 1.0514622242382672
Current ro 0.5465330578253433 - Iteration 1, points 42
Current ro 0.2759044842552674 - Iteration 2, points 162
Current ro 0.13828317354716765 - Iteration 3, points 642
shell with 642 particles
ps input 0, recomputed -0.6123724356957945 , sideN-1.0
gen inner particles in colloid
0.13828317354716765 ['0.008000000000000002', '0.008000000000000002', '0.008000000000000002', '0.008000000000000002']
Current ro 1.0514622242382672
Current ro 0.5465330578253433 - Iteration 1, points 654
Current ro 0.2759044842552674 - Iteration 2, points 654
Current ro 0.13828317354716765 - Iteration 3, points 654
solvent to delete 524, with dh 0.2
15101 0 15101


In [6]:
# Running the code to compute mobility of the virus
codefile = 'RigidMultiblobsWall/multi_bodies/multi_bodies_utilities.py'
for i in range(0,len(sids)):
    f = open("RigidMultiblobsWall/multi_bodies/inputfile_body_mobilityVirus.dat", 'r')
    output = open("RigidMultiblobsWall/multi_bodies/inputs/inputfile_body_mobilityVirus%s.dat"%sids[i], 'w')
    for line in f:
        l = line
        if re.match('blob_radius', l) != None:
            output.write("blob_radius %s \n"%(ros[i]/2))
        else:    
            output.write(l.rstrip() + '\n')
    f.close()
    output.close()
    infile = 'RigidMultiblobsWall/multi_bodies/inputs/inputfile_body_mobilityVirus%s.dat' % sids[i]
    os.system("python %s --input-file %s --idvir %s" %(codefile,infile,sids[i]))

searching functions in path  RigidMultiblobsWall/
cannot import name 'visit_writer_interface' from 'visit' (/home/dmoreno/RigidMultiblobsWall/visit/__init__.py)
Creating structures =  RigidMultiblobsWall/multi_bodies/Structures/blob.clones
Time to compute body mobility = 0.2997763156890869



# End


In [7]:
ros2 = []
for i in sids:
    infFile = 'RigidMultiblobsWall/multi_bodies/data/info%s' % (i)        
    inf = pd.read_csv(infFile,sep=' ',index_col=0,header=None).to_dict()#np.loadtxt(f)

In [8]:
##this is an example of how to extrac the data from the files. this can iterate over all the sids or only those
#of interest for example sids  = [1,2,3,4,5] for only index 1 to 5

Mts = []
Mrs = []
Mcs = []
RtX  = []
RrotX = []
RtY  = []
RrotY = []
RtZ  = []
RrotZ = []
ros2 = []
Rtprom = []
Rrotprom = []

for i in sids:
        dFile = 'RigidMultiblobsWall/multi_bodies/data/virusMob%s.body_mobility.dat' % (i)
        infFile = 'RigidMultiblobsWall/multi_bodies/data/info%s' % (i)
        dat = pd.read_csv(dFile,sep='  ',header=None,engine='python')#np.loadtxt(f)
        inf = pd.read_csv(infFile,sep=' ',index_col=0,header=None,engine='python').to_dict()#np.loadtxt(f)
        ros2.append(inf[1]["ro"])
        mt,mr,mc = M(dat)  ##separate the matrix in three translation, rotation and coupling
        valt,vect= np.linalg.eig(mt.T)  ##diagonalize each of the matrix
        valr,vecr= np.linalg.eig(mr.T) 
        valc,vecc= np.linalg.eig(mc.T) 
        Mts.append(mt)
        Mrs.append(mr)
        Mcs.append(mc)
#Axis X and Z are swiched from ellipsoids
        RtX.append(1*xi/(mt[0,0]*6*np.pi))
        #RrotX.append(1*xi/(mr[0,0]*8*np.pi)**(1/3.))
        RtY.append(1*xi/(mt[1,1]*6*np.pi))
        #RrotY.append(1/(mr[1,1]*8*np.pi)**(1/3.))
        RtZ.append(1*xi/(mt[2,2]*6*np.pi))
        #RrotZ.append(1/(mr[2,2]*8*np.pi)**(1/3.))
        Rtprom.append((1/(((mt[0,0]+mt[1,1]+mt[2,2])/3)*6*np.pi)))
        #Rrotprom.append(1/(((mr[0,0]+mr[1,1]+mr[2,2])/3)*8*np.pi)**(1/3.))
Mts = np.array(Mts)
Mrs = np.array(Mrs)
Mcs = np.array(Mcs)
ros2 = np.array(ros2,dtype=float)
Rtprom
Mts,Mrs,Mcs,RtX

(array([[[ 5.18112964e-02, -1.86268392e-18, -5.83020066e-18],
         [ 1.86268392e-18,  5.18112964e-02, -1.09913962e-17],
         [ 5.83020066e-18, -4.05850465e-17,  5.18112964e-02]]]),
 array([[[ 3.72024561e-02, -9.04102722e-19,  2.31587995e-17],
         [-1.46356684e-17,  3.72024561e-02,  4.31063647e-17],
         [-1.89891342e-17, -3.24899880e-18,  3.72024561e-02]]]),
 array([[[-2.86155004e-17, -5.04274958e-17,  7.14153014e-17],
         [-8.12282936e-17, -1.07831461e-16,  9.78042520e-17],
         [-1.30108287e-17, -5.31487226e-17,  5.92277066e-17]]]),
 [1.0239397855434618])